In [107]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import json
import re
import os
 

In [94]:
path1 = "./Movie_Poster_Metadata/groundtruth"
temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
path2 = "./Movie_Poster_Metadata/updated_groundtruth"

### Reading the input file and creating a clean one

In [126]:
dir_list = os.listdir(path1)
 

if not os.path.exists(temp_path):
  os.makedirs(temp_path)

if not os.path.exists(path2):
  os.makedirs(path2)

    
for file_name in dir_list:
    
    with open(path1+'/'+file_name) as file1:

        temp_file = open(temp_path+'/'+file_name,'w')

        temp_file.write('[')
        
        for line in file1.readlines():

            line = line.replace("}\n","},\n")
            
            # reading all lines that begin 
            # with "  "_id""
            y = re.findall("^  \"_id\"", line)
            if not y:
                temp_file.write(line)
                            
        
#         file2.write(']')
    file1.close()
    temp_file.close()


In [124]:
dir_list = os.listdir(temp_path)
 
for file_name in dir_list:
    
    with open(temp_path+'/'+file_name) as temp_file:
    
        file2 = open(path2+'/'+file_name,'w')

        lines = temp_file.readlines()
        lines = lines[:-1]

        file2.writelines(lines)
        file2.write('}]')

    temp_file.close()
    file2.close()

shutil.rmtree(temp_path)

### Function to append all the json objects into dataframe 

In [125]:
movies_df = pd.DataFrame()

df1 = pd.read_json(path2+'/1980.txt')

df1 = df1[['imdbID','Director','Genre','imdbRating']]

print(df1.shape)
pd.concat([movies_df,df1], ignore_index=True)

df2 = pd.read_json(path2+'/1981.txt')

df2 = df2[['imdbID','Director','Genre','imdbRating']]

pd.concat([movies_df,df2], ignore_index=True)

print(movies_df.dtypes)
print(movies_df.head(20))
print(movies_df.shape)

df1 = pd.concat([df1,df2], ignore_index=True)

print(df1.dtypes)
print(df1.head(20))
print(df1.shape)


(97, 4)
Series([], dtype: object)
Empty DataFrame
Columns: []
Index: []
(0, 0)
imdbID        object
Director      object
Genre         object
imdbRating    object
dtype: object
       imdbID                                  Director  \
0   tt0080684                            Irvin Kershner   
1   tt0081562                            Sidney Poitier   
2   tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
3   tt0080377                            Buddy Van Horn   
4   tt0081375                              Howard Zieff   
5   tt0080549                             Michael Apted   
6   tt0081529                               Hal Needham   
7   tt0080453                            Randal Kleiser   
8   tt0080455                               John Landis   
9   tt0081283                            Robert Redford   
10  tt0081353                             Robert Altman   
11  tt0081696                             James Bridges   
12  tt0081505                           Stanley Kubrick 

In [122]:


dir_list = os.listdir(path2)

movies_df = pd.DataFrame()

for file_name in dir_list:    

    try:
        df = pd.read_json(path2+'/'+file_name)
        df = df[['imdbID','Director','Genre','imdbRating']]
        movies_df = pd.concat([movies_df,df], ignore_index=True)

    except:
        print(file_name)
        
print(movies_df.dtypes)
print(movies_df.head(20))
print(movies_df.shape)

1982.txt
1983.txt
1984.txt
1985.txt
1986.txt
1987.txt
1988.txt
1989.txt
1990.txt
1991.txt
1992.txt
1993.txt
1994.txt
1995.txt
1996.txt
1997.txt
1998.txt
1999.txt
2000.txt
2001.txt
2002.txt
2003.txt
2004.txt
2005.txt
2006.txt
2007.txt
2008.txt
2009.txt
2010.txt
2011.txt
2012.txt
2013.txt
2014.txt
2015.txt
imdbID        object
Director      object
Genre         object
imdbRating    object
dtype: object
       imdbID                                  Director  \
0   tt0080684                            Irvin Kershner   
1   tt0081562                            Sidney Poitier   
2   tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
3   tt0080377                            Buddy Van Horn   
4   tt0081375                              Howard Zieff   
5   tt0080549                             Michael Apted   
6   tt0081529                               Hal Needham   
7   tt0080453                            Randal Kleiser   
8   tt0080455                               John Landis   
9   tt